<a href="https://colab.research.google.com/github/bonillahermes/Data_Science_Projects/blob/main/EDA10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de Gomas

## Librerías Necesarias

In [ ]:
# install.packages("readr")
# install.packages("readxl")
# install.packages("ggplot2")
# install.packages("dplyr")
install.packages("nortest")
install.packages("Rcpp")
install.packages("mvtnorm")
install.packages("multcompView")
install.packages("SuppDists")
install.packages("kSamples")
install.packages("BWStest")
install.packages("corrplot")
install.packages("psych")

In [ ]:
#library(readr)
library(corrplot)
library(readxl)
library(ggplot2)
library(tidyverse)
library(nortest)
library(psych)

## Cargar base de datos

In [ ]:
# Especifica la ruta del archivo xlsx, reemplaza "ruta/del/archivo/Base.xlsx" con la ubicación real de tu archivo
ruta_archivo <- "Base.xlsx"

# Lee el archivo xlsx y almacena los datos en un objeto llamado "datos"
datos <- read_excel(ruta_archivo)

# Muestra las primeras filas del conjunto de datos para verificar la importación
head(datos)

In [ ]:
# Convertir variables a factor
datos$`Formulación` <- as.factor(datos$`Formulación`)
datos$Carga <- as.factor(datos$Carga)


## Estadísticas Descriptivas Global

En cuanto a la fuerza (Force), los datos muestran una variabilidad considerable, con valores oscilando desde -0.028 hasta 32.616, con una media de 10.585. La distancia (Distance) varía entre 0 y 1, destacando una mediana de 0.5. La deformación (Deformacion) presenta valores comprendidos entre 0 y 0.16667, con una mediana de 0.08333. La tensión (Tension) se caracteriza por su variabilidad, con valores desde -3.084e-05 hasta 3.592e-02, y una mediana de 1.001e-02

In [ ]:
# Selecciona las VARIABLES de interés
columnas_interes <- datos[, c("Force", "Distance", "Deformacion", "Tension")]

# Calcula estadísticas descriptivas
estadisticas_descriptivas <- summary(columnas_interes)

# Muestra las estadísticas descriptivas
print(estadisticas_descriptivas)

Para la fuerza (Force), se observa un coeficiente de variación del 64.60%, indicando una alta dispersión en comparación con la media. La distancia (Distance) y la deformación (Deformacion) exhiben coeficientes de variación del 57.79%, reflejando niveles similares de variabilidad relativa. La tensión (Tension) presenta un coeficiente de variación del 65.06%, indicando una dispersión considerable en relación con la media.

In [ ]:
# Calcula el coeficiente de variación para cada columna
coeficiente_variacion <- sapply(columnas_interes, function(x) sd(x) / mean(x))

# Muestra los resultados
print(coeficiente_variacion)


Todas las variables, excepto fuerza (Force) presentan una distribución uniforme.m

In [ ]:
# Ajusta el tamaño del gráfico
options(repr.plot.width = 10, repr.plot.height = 8)

# Crea un layout de gráficos con 2 filas y 2 columnas
par(mfrow = c(2, 2))

# Histogramas con ggplot2
ggplot(datos, aes(x = Force, fill = "lightblue")) +
  geom_histogram(binwidth = 5, color = "black", alpha = 0.7) +
  labs(title = "Histograma de Force", x = "Force", y = "Frecuencia") +
  theme_minimal() +
  theme(axis.text = element_text(size = 12), axis.title = element_text(size = 14))

ggplot(datos, aes(x = Distance, fill = "lightgreen")) +
  geom_histogram(binwidth = 5, color = "black", alpha = 0.7) +
  labs(title = "Histograma de Distance", x = "Distance", y = "Frecuencia") +
  theme_minimal() +
  theme(axis.text = element_text(size = 12), axis.title = element_text(size = 14))

ggplot(datos, aes(x = Deformacion, fill = "lightcoral")) +
  geom_histogram(binwidth = 5, color = "black", alpha = 0.7) +
  labs(title = "Histograma de Deformacion", x = "Deformacion", y = "Frecuencia") +
  theme_minimal() +
  theme(axis.text = element_text(size = 12), axis.title = element_text(size = 14))

ggplot(datos, aes(x = Tension, fill = "lightsalmon")) +
  geom_histogram(binwidth = 5, color = "black", alpha = 0.7) +
  labs(title = "Histograma de Tension", x = "Tension", y = "Frecuencia") +
  theme_minimal() +
  theme(axis.text = element_text(size = 12), axis.title = element_text(size = 14))

La fuerza y la tensión presentan algunos datos atípicos.

In [ ]:
# Boxplots con ggplot2
ggplot(datos, aes(x = "Force", y = Force, fill = "lightblue")) +
  geom_boxplot() +
  labs(title = "Boxplot de Force", x = "", y = "Force") +
  theme_minimal() +
  theme(axis.text = element_text(size = 12), axis.title = element_text(size = 14))

ggplot(datos, aes(x = "Distance", y = Distance, fill = "lightgreen")) +
  geom_boxplot() +
  labs(title = "Boxplot de Distance", x = "", y = "Distance") +
  theme_minimal() +
  theme(axis.text = element_text(size = 12), axis.title = element_text(size = 14))

ggplot(datos, aes(x = "Deformacion", y = Deformacion, fill = "lightcoral")) +
  geom_boxplot() +
  labs(title = "Boxplot de Deformacion", x = "", y = "Deformacion") +
  theme_minimal() +
  theme(axis.text = element_text(size = 12), axis.title = element_text(size = 14))

ggplot(datos, aes(x = "Tension", y = Tension, fill = "lightsalmon")) +
  geom_boxplot() +
  labs(title = "Boxplot de Tension", x = "", y = "Tension") +
  theme_minimal() +
  theme(axis.text = element_text(size = 12), axis.title = element_text(size = 14))

## Pruebas de Normalidad


La prueba de Anderson-Darling es una prueba estadística no paramétrica que evalúa la hipótesis nula de que una muestra proviene de una población con una distribución específica (en este caso, se asume que la distribución es normal). El estadístico Anderson-Darling (Anderson_Darling_Stat) cuantifica la discrepancia entre la distribución observada y la distribución teórica (en este caso, la distribución normal). Cuanto mayor sea el estadístico, mayor será la discrepancia. El p-valor asociado (Anderson_Darling_P_Value) indica la probabilidad de obtener un estadístico Anderson-Darling tan extremo como el observado, asumiendo que la distribución es normal. En este contexto, los p-valores extremadamente bajos (3.7e-24) para todas las variables (Force, Distance, Deformacion y Tension) sugieren evidencia significativa para rechazar la hipótesis nula de normalidad. Por lo tanto, se concluye que las variables no siguen una distribución normal,

In [ ]:
# Pruebas de normalidad para Force
ad_test_force <- ad.test(datos$Force)

# Pruebas de normalidad para Distance
ad_test_distance <- ad.test(datos$Distance)

# Pruebas de normalidad para Deformacion
ad_test_deformacion <- ad.test(datos$Deformacion)

# Pruebas de normalidad para Tension
ad_test_tension <- ad.test(datos$Tension)

# Muestra los resultados
resultados_normalidad <- data.frame(
  Variable = c("Force", "Distance", "Deformacion", "Tension"),
  Anderson_Darling_Stat = c(ad_test_force$statistic, ad_test_distance$statistic, ad_test_deformacion$statistic, ad_test_tension$statistic),
  Anderson_Darling_P_Value = c(ad_test_force$p.value, ad_test_distance$p.value, ad_test_deformacion$p.value, ad_test_tension$p.value)
)

print(resultados_normalidad)

## Fuerza según Carga y Formulacion



Se filtra los datos, aquellos sin carga quedarán en la base `datos_carga_0` y los cargados en `datos_carga_1`.

In [ ]:
# Filtra los datos para incluir solo aquellos con carga 0
datos_carga_0 <- datos[datos$Carga == 0, ]

# Filtra los datos para incluir solo aquellos con carga 1
datos_carga_1 <- datos[datos$Carga == 1, ]

### Agrupamiento por Carga

El análisis de la variable de Fuerza según la carga (0 sin carga y 1 con carga) revela patrones distintivos en ambas condiciones. Para los casos sin carga, la fuerza varía desde -0.027 hasta 32.616, con una mediana de 8.253 y una media de 10.308. El rango intercuartílico (IQR) se extiende desde 4.231 hasta 15.325, resaltando una variabilidad moderada en los datos. En comparación, los casos con carga exhiben una fuerza que oscila desde -0.028 hasta 30.878, con una mediana de 10.396 y una media de 10.930. El IQR va desde 5.440 hasta 15.778, indicando una distribución de fuerza similar a los casos sin carga, aunque con valores ligeramente más altos. En general, estos resultados sugieren que, aunque hay similitudes en la distribución de la fuerza entre las dos condiciones, la presencia de carga podría asociarse con valores ligeramente superiores.

In [ ]:
# Resumen estadístico por grupo
summary_by_carga <- tapply(datos$Force, datos$Carga, summary)

# Mostrar el resumen
print(summary_by_carga)

### Test de Diferencias en las medianas

La prueba de Wilcoxon rank sum test, también conocida como prueba U de Mann-Whitney, se realizó para comparar las medianas de la variable Force entre los dos grupos definidos por la variable Carga (0 sin carga y 1 con carga). Los resultados indican un estadístico W de 39153745 y un p-valor extremadamente bajo (< 2.2e-16), lo que lleva a rechazar la hipótesis nula de que no hay diferencia en la ubicación (mediana) entre los dos grupos. En otras palabras, hay evidencia significativa para afirmar que hay un cambio en la mediana de la variable Force entre los casos con carga y sin carga. Gráficamente se puede concluir que la fuerza empleada es mayor en las gomas cargadas.

In [ ]:
# Boxplot de Force según la variable Carga
ggplot(datos, aes(x = Carga, y = Force, fill = Carga)) +
  geom_boxplot() +
  labs(title = "Boxplot de Force según la Carga", x = "Carga", y = "Force") +
  theme_minimal() +
  theme(axis.text = element_text(size = 12), axis.title = element_text(size = 14))

In [ ]:
# Realiza la prueba de Wilcoxon-Mann-Whitney
wilcox.test(Force ~ Carga, data = datos)

### Agrupamiento por Formulación

El análisis de la fuerza según el tipo de formulación (identificado por los códigos numéricos 11, 12, 13, 21, 22, 23, 31, 32, 33) revela variabilidad en las distribuciones de fuerza entre las diferentes formulaciones. Para la formulación 11, la fuerza varía desde -0.028 hasta 29.654, con una mediana de 11.384 y una media de 11.963. En comparación, la formulación 12 exhibe una fuerza que oscila entre -0.013 y 32.616, con una mediana de 9.253 y una media de 10.386. Similar variabilidad se observa en las otras formulaciones, destacando diferentes medianas y promedios.

In [ ]:
# Resumen estadístico por grupo
summary_by_formulacion <- tapply(datos$Force, datos$`Formulación`, summary)

# Mostrar el resumen
print(summary_by_formulacion)

In [ ]:
# Boxplot de Force según la variable Formulacion
ggplot(datos, aes(x = Formulación, y = Force, fill = Formulación)) +
  geom_boxplot() +
  labs(title = "Boxplot de Force según la Formulación", x = "Formulación", y = "Force") +
  theme_minimal() +
  theme(axis.text = element_text(size = 12), axis.title = element_text(size = 14))

### Test de diferencias de medianas


La prueba de Kruskal-Wallis fue realizada para comparar las medianas de la variable Force entre las diferentes formulaciones (identificadas por los códigos numéricos 11, 12, 13, 21, 22, 23, 31, 32, 33). El estadístico de chi-cuadrado de Kruskal-Wallis resultó en 908.1 con 8 grados de libertad, y el p-valor asociado es extremadamente bajo (< 2.2e-16). Este resultado lleva a rechazar la hipótesis nula de que las medianas son iguales en todos los grupos, indicando que al menos una de las formulaciones difiere significativamente en términos de la fuerza. En consecuencia, se concluye que hay diferencias significativas en la fuerza entre las distintas formulaciones.

In [ ]:
# Realiza la prueba de Kruskal-Wallis
kruskal.test(Force ~ Formulación, data = datos)

### Pruebas Post hoc

Las comparaciones pareadas utilizando la prueba de Wilcoxon rank sum test con corrección de continuidad indican diferencias significativas en la fuerza entre diferentes formulaciones. Cada celda de la tabla proporciona el p-valor correspondiente a la comparación entre las formulaciones, un p-valor menor a 0.05 indica que las diferencias entre las medianas son significativas, por ejemplo:
- La comparación entre las formulaciones 11 y 12 muestra un p-valor de 4.9e-07, indicando una diferencia significativa en la fuerza entre estas formulaciones.
- El p-valor de 1.0000 en la celda (11, 23) indica que no hay diferencia significativa entre la formulación 11 y 23 (lo mismo para las diagonales).

In [ ]:
# Realiza pruebas post hoc de Wilcoxon con ajuste de p-valores
pruebas_post_hoc <- pairwise.wilcox.test(datos$Force, datos$`Formulación`, p.adj = "bonferroni")

# Muestra las pruebas post hoc
print(pruebas_post_hoc)


### Fuerza por formulación agrunpando por carga 0

La interpretación de los siguientes resultados se realizan de manera similar a los anteriores casos.
Se puede observar que los datos sin carga, la fuerza es mayor en la formulación 12 aunque es muy dispersa. La formulación 33 presenta la menor fuerza y esta es más concentrada.

In [ ]:
# Boxplot de Force según la variable Formulación solo para carga 0
ggplot(datos_carga_0, aes(x = Formulación, y = Force, fill = Formulación)) +
  geom_boxplot() +
  labs(title = "Boxplot de Force según la Formulación (Carga 0)", x = "Formulación", y = "Force") +
  theme_minimal() +
  theme(axis.text = element_text(size = 12), axis.title = element_text(size = 14))


La prueba de Kruskal-Wallis fue realizada para comparar las medianas de la variable Force entre las diferentes formulaciones sin carga. El estadístico de chi-cuadrado de Kruskal-Wallis resultó en 2447.6 con 8 grados de libertad, y el p-valor asociado es extremadamente bajo (< 2.2e-16). Este resultado lleva a rechazar la hipótesis nula de que las medianas son iguales en todos los grupos, indicando que al menos una de las formulaciones difiere significativamente en términos de la fuerza. En consecuencia, se concluye que hay diferencias significativas en la fuerza entre las distintas formulaciones.

In [ ]:
# Realiza la prueba de Kruskal-Wallis
kruskal.test(Force ~ Formulación, data = datos_carga_0)

In [ ]:
# Realiza pruebas post hoc de Wilcoxon con ajuste de p-valores
pruebas_post_hoc0 <- pairwise.wilcox.test(datos$Force, datos_carga_0$`Formulación`, p.adj = "bonferroni")

# Muestra las pruebas post hoc
print(pruebas_post_hoc0)

### Fuerza por formulación agrunpando por carga 1

Ajustando la carga, se pueden notar algunas diferencias. La formulación 33 con carga ahora presenta el mayor valor mediano en fuerza y la formulación 12 ahora presenta el menor valor.

In [ ]:
# Boxplot de Force según la variable Formulación solo para carga 1
ggplot(datos_carga_1, aes(x = Formulación, y = Force, fill = Formulación)) +
  geom_boxplot() +
  labs(title = "Boxplot de Force según la Formulación (Carga 1)", x = "Formulación", y = "Force") +
  theme_minimal() +
  theme(axis.text = element_text(size = 12), axis.title = element_text(size = 14))


El estadístico de chi-cuadrado de Kruskal-Wallis resultó en 785.86 con 8 grados de libertad, y el p-valor asociado es extremadamente bajo (< 2.2e-16). Este resultado lleva a rechazar la hipótesis nula de que las medianas son iguales en todos los grupos, indicando que al menos una de las formulaciones difiere significativamente en términos de la fuerza. En consecuencia, se concluye que hay diferencias significativas en la fuerza entre las distintas formulaciones.

In [ ]:
# Realiza la prueba de Kruskal-Wallis
kruskal.test(Force ~ Formulación, data = datos_carga_1)

In [ ]:
# Realiza pruebas post hoc de Wilcoxon con ajuste de p-valores
pruebas_post_hoc1 <- pairwise.wilcox.test(datos$Force, datos_carga_1$`Formulación`, p.adj = "bonferroni")

# Muestra las pruebas post hoc
print(pruebas_post_hoc1)

## Distancia según carga y formulación

### Agrupamiento por Carga


Los resultados para la variable Distance, analizados según la presencia o ausencia de carga (Carga 0 y Carga 1), revelan similitudes notables en la distribución de distancias entre ambos grupos. En ambos casos, la variable presenta una mediana de 0.5, sugiriendo que al menos el 50% de las observaciones tienen una distancia de 0.5 o menos. Los valores mínimos, cuartiles, mediana, media, tercer cuartil y máximo son prácticamente idénticos en ambos grupos, lo que indica una consistencia en las características centrales y de dispersión de la variable Distance, independientemente de la presencia de carga.

In [ ]:
# Resumen estadístico por grupo
summary_by_carga <- tapply(datos$Distance, datos$Carga, summary)

# Mostrar el resumen
print(summary_by_carga)

In [ ]:
# Boxplot de Distance según la variable Carga
ggplot(datos, aes(x = Carga, y = Distance, fill = Carga)) +
  geom_boxplot() +
  labs(title = "Boxplot de Distance según la Carga", x = "Carga", y = "Distance") +
  theme_minimal() +
  theme(axis.text = element_text(size = 12), axis.title = element_text(size = 14))

La prueba de Wilcoxon rank sum test, realizada para comparar las medianas de la variable Distance entre los dos grupos definidos por la variable Carga (0 sin carga y 1 con carga), indica un estadístico W de 42672146 y un p-valor de 0.9735. Este alto p-valor sugiere que no hay evidencia significativa para rechazar la hipótesis nula de que no hay un cambio en la ubicación (mediana) entre los dos grupos. En otras palabras, no se encuentran diferencias significativas en la mediana de la variable Distance entre los casos con carga y sin carga, según los resultados de esta prueba.

In [ ]:
# Realiza la prueba de Wilcoxon-Mann-Whitney
wilcox.test(Distance ~ Carga, data = datos)

### Agrupamiento por Formulario

Debido a la uniformidad de la variable Distance, es evidente las similitudes en la distribución de distancias entre las formulaciones.

In [ ]:
# Resumen estadístico por grupo
summary_by_formulacion <- tapply(datos$Distance, datos$`Formulación`, summary)

# Mostrar el resumen
print(summary_by_formulacion)

In [ ]:
# Boxplot de Force según la variable Formulacion
ggplot(datos, aes(x = Formulación, y = Distance, fill = Formulación)) +
  geom_boxplot() +
  labs(title = "Boxplot de Distance según la Formulación", x = "Formulación", y = "Distance") +
  theme_minimal() +
  theme(axis.text = element_text(size = 12), axis.title = element_text(size = 14))

Tanto la prueba de Kruskal-Wallis y las pruebas post hoc, muestran que no existen diferencias significativas entre los valores medianos entre los grupos y por cada pareja de formulación.

In [ ]:
# Realiza la prueba de Kruskal-Wallis
kruskal.test(Distance ~ Formulación, data = datos)

In [ ]:
# Realiza pruebas post hoc de Wilcoxon con ajuste de p-valores
pruebas_post_hoc <- pairwise.wilcox.test(datos$Distance, datos$`Formulación`, p.adj = "bonferroni")

# Muestra las pruebas post hoc
print(pruebas_post_hoc)

### Distancia por formulación agrunpando por carga 0

In [ ]:
# Boxplot de Distance según la variable Formulación solo para carga 0
ggplot(datos_carga_0, aes(x = Formulación, y = Distance, fill = Formulación)) +
  geom_boxplot() +
  labs(title = "Boxplot de Distance según la Formulación (Carga 0)", x = "Formulación", y = "Distance") +
  theme_minimal() +
  theme(axis.text = element_text(size = 12), axis.title = element_text(size = 14))

La prueba de Kruskall-wallis revela que no existe diferencias significativas entre los valores medianos en la distancia sin carga con estadístico de prueba de 0.00271 y un p-valor muyr cercano a 1.

In [ ]:
# Realiza la prueba de Kruskal-Wallis
kruskal.test(Distance ~ Formulación, data = datos_carga_0)

### Distancia por formulación agrunpando por carga 1

In [ ]:
# Boxplot de Distance según la variable Formulación solo para carga 1
ggplot(datos_carga_1, aes(x = Formulación, y = Distance, fill = Formulación)) +
  geom_boxplot() +
  labs(title = "Boxplot de Distance según la Formulación (Carga 1)", x = "Formulación", y = "Distance") +
  theme_minimal() +
  theme(axis.text = element_text(size = 12), axis.title = element_text(size = 14))

La prueba de Kruskall-wallis revela que no existe diferencias significativas entre los valores medianos en la distancia con carga con estadístico de prueba de 0.015913 y un p-valor muyr cercano a 1.

In [ ]:
# Realiza la prueba de Kruskal-Wallis
kruskal.test(Distance ~ Formulación, data = datos_carga_1)

## Deformación

### Agrupamiento por Carga

Los resultados para la variable Deformacion, analizados en función de la presencia o ausencia de carga (Carga 0 y Carga 1), indican similitudes sustanciales en la distribución de la deformación entre ambos grupos. Para el grupo sin carga (Carga 0), la deformación muestra una mediana y media de 0.08333, denotando que el 50% de las observaciones tienen una deformación de 0.08333 o menos, y el promedio de todas las deformaciones es 0.08330. Los valores mínimo y máximo son 0 y 0.16667, respectivamente. En el grupo con carga (Carga 1), los valores estadísticos son prácticamente idénticos, indicando coherencia en las características centrales y de dispersión de la variable Deformacion entre los dos grupos.

In [ ]:
# Resumen estadístico por grupo
summary_by_carga <- tapply(datos$Deformacion, datos$Carga, summary)

# Mostrar el resumen
print(summary_by_carga)

In [ ]:
# Boxplot de Distance según la variable Carga
ggplot(datos, aes(x = Carga, y = Deformacion, fill = Carga)) +
  geom_boxplot() +
  labs(title = "Boxplot de Deformacion según la Carga", x = "Carga", y = "Deformacion") +
  theme_minimal() +
  theme(axis.text = element_text(size = 12), axis.title = element_text(size = 14))

La prueba de Wilcoxon rank sum test para comparar las medianas de la variable Deformacion entre los dos grupos definidos por la variable Carga (0 sin carga y 1 con carga) revela un estadístico W de 42672150 y un p-valor de 0.9736. Este p-valor alto sugiere que no hay evidencia significativa para rechazar la hipótesis nula de que no hay un cambio en la ubicación (mediana) entre los dos grupos. En otras palabras, no se encuentran diferencias significativas en la mediana de la variable Deformacion entre los casos con carga y sin carga, según los resultados de esta prueba. Esto respalda la observación anterior de similitudes sustanciales en la distribución de la deformación entre ambos grupos.

In [ ]:
# Realiza la prueba de Wilcoxon-Mann-Whitney
wilcox.test(Deformacion ~ Carga, data = datos)

### Agrupamiento por Formulación

Los resultados para la variable Deformacion, desglosados por los diferentes tipos de formulación identificados por los códigos numéricos (11, 12, 13, 21, 22, 23, 31, 32, 33), muestran consistencias notables en las características centrales de la deformación para cada tipo de formulación. En general, la mediana y media de la deformación son de 0.08333, indicando que el 50% de las observaciones tienen una deformación de 0.08333 o menos, y el promedio de todas las deformaciones es muy cercano a este valor. Los valores mínimo y máximo son 0 y 0.16667, respectivamente, en todos los casos, denotando que la variable Deformacion se encuentra en un rango específico en todas las formulaciones.

In [ ]:
# Resumen estadístico por grupo
summary_by_formulacion <- tapply(datos$Deformacion, datos$`Formulación`, summary)

# Mostrar el resumen
print(summary_by_formulacion)

In [ ]:
# Boxplot de Deformacion según la variable Formulacion
ggplot(datos, aes(x = Formulación, y = Deformacion, fill = Formulación)) +
  geom_boxplot() +
  labs(title = "Boxplot de Deformacion según la Formulación", x = "Formulación", y = "Deformacion") +
  theme_minimal() +
  theme(axis.text = element_text(size = 12), axis.title = element_text(size = 14))

La prueba de Kruskal-Wallis para comparar las medianas de la variable Deformacion entre las diferentes formulaciones (códigos numéricos 11, 12, 13, 21, 22, 23, 31, 32, 33) muestra un estadístico de chi-cuadrado de 0.0087417 con 8 grados de libertad y un p-valor de 1. Este alto p-valor sugiere que no hay evidencia significativa para rechazar la hipótesis nula de que las medianas son iguales en todos los grupos. En otras palabras, no se encuentran diferencias significativas en la mediana de la variable Deformacion entre las diferentes formulaciones, según los resultados de esta prueba. Estos hallazgos respaldan la observación anterior de consistencias notables en las características centrales de la deformación entre los diferentes tipos de formulación.

In [ ]:
# Realiza la prueba de Kruskal-Wallis
kruskal.test(Deformacion ~ Formulación, data = datos)

### Deformación por formulación agrunpando por carga 0

Los gráficos para la deformación con o sin carga, muestran comportamientos muy similares. La prueba de Kruskal-Wallis confirma lo anterior, pues dado que el p-valor en ambas grupos es muy cercano 1 muestra que sin importar la carga, no hay diferencias significativas entre los tipos de formulaciones.

In [ ]:
# Boxplot de Deformacion según la variable Formulación solo para carga 0
ggplot(datos_carga_0, aes(x = Formulación, y = Deformacion, fill = Formulación)) +
  geom_boxplot() +
  labs(title = "Boxplot de Deformacion según la Formulación (Carga 0)", x = "Formulación", y = "Deformacion") +
  theme_minimal() +
  theme(axis.text = element_text(size = 12), axis.title = element_text(size = 14))

In [ ]:
# Realiza la prueba de Kruskal-Wallis
kruskal.test(Deformacion ~ Formulación, data = datos_carga_0)

### Deformación por formulación agrunpando por carga 1

In [ ]:
# Boxplot de Force según la variable Formulación solo para carga 1
ggplot(datos_carga_1, aes(x = Formulación, y = Deformacion, fill = Formulación)) +
  geom_boxplot() +
  labs(title = "Boxplot de Deformacion según la Formulación (Carga 1)", x = "Formulación", y = "Deformacion") +
  theme_minimal() +
  theme(axis.text = element_text(size = 12), axis.title = element_text(size = 14))

In [ ]:
# Realiza la prueba de Kruskal-Wallis
kruskal.test(Deformacion ~ Formulación, data = datos_carga_1)

## Tensión

### Agrupamiento por carga

Los resultados para la variable Tension, evaluados en función de la presencia o ausencia de carga (Carga 0 y Carga 1), revelan semejanzas notables en la distribución de la tensión entre ambos grupos. Tanto para los casos con carga como para aquellos sin carga, la tensión exhibe patrones similares, con mínimos, cuartiles, mediana, media, tercer cuartil y máximo prácticamente idénticos. La tensión oscila desde valores negativos cercanos a cero hasta valores positivos, siendo la media alrededor de 1.122e-02 y 1.204e-02 para los grupos sin carga y con carga, respectivamente.

In [ ]:
# Resumen estadístico por grupo
summary_by_carga <- tapply(datos$Tension, datos$Carga, summary)

# Mostrar el resumen
print(summary_by_carga)

In [ ]:
# Boxplot de Distance según la variable Carga
ggplot(datos, aes(x = Carga, y = Tension, fill = Carga)) +
  geom_boxplot() +
  labs(title = "Boxplot de Tension según la Carga", x = "Carga", y = "Tension") +
  theme_minimal() +
  theme(axis.text = element_text(size = 12), axis.title = element_text(size = 14))

La prueba de Wilcoxon rank sum test para comparar las medianas de la variable Tension entre los dos grupos definidos por la variable Carga (0 sin carga y 1 con carga) muestra un estadístico W de 38645356 y un p-valor extremadamente bajo (< 2.2e-16). Este resultado lleva a rechazar la hipótesis nula de que no hay cambio en la ubicación (mediana) entre los dos grupos, indicando que hay diferencias significativas en la mediana de la variable Tension entre los casos con carga y sin carga. En otras palabras, la presencia de carga influye de manera significativa en la distribución de la tensión, según los resultados de esta prueba no paramétrica.

In [ ]:
# Realiza la prueba de Wilcoxon-Mann-Whitney
wilcox.test(Tension ~ Carga, data = datos)

### Agrupamiento por formulario

Los resultados para la variable Tension, desglosados por los diferentes tipos de formulación identificados por los códigos numéricos (11, 12, 13, 21, 22, 23, 31, 32, 33), reflejan variaciones en las características centrales de la tensión para cada tipo de formulación. En términos generales, para todas las formulaciones, la tensión oscila desde valores negativos cercanos a cero hasta valores positivos. La mediana y la media de la tensión tienden a incrementarse de 11 a 33. Por ejemplo, la formulación 11 presenta una mediana y media de 1.254e-02 y 1.318e-02, respectivamente, mientras que la formulación 33 tiene valores de 6.716e-03 y 9.919e-03. Los cuartiles y valores máximos también varían entre las formulaciones.

In [ ]:
# Resumen estadístico por grupo
summary_by_formulacion <- tapply(datos$Tension, datos$`Formulación`, summary)

# Mostrar el resumen
print(summary_by_formulacion)

In [ ]:
# Boxplot de Tension según la variable Formulacion
ggplot(datos, aes(x = Formulación, y = Tension, fill = Formulación)) +
  geom_boxplot() +
  labs(title = "Boxplot de Tension según la Formulación", x = "Formulación", y = "Tension") +
  theme_minimal() +
  theme(axis.text = element_text(size = 12), axis.title = element_text(size = 14))

La prueba de Kruskal-Wallis para comparar las medianas de la variable Tension entre las diferentes formulaciones (códigos numéricos 11, 12, 13, 21, 22, 23, 31, 32, 33) muestra un estadístico de chi-cuadrado de 1004 con 8 grados de libertad y un p-valor extremadamente bajo (< 2.2e-16). Este resultado sugiere que hay diferencias significativas en las medianas de la variable Tension entre al menos dos de los grupos de formulaciones. En otras palabras, la formulación influye de manera significativa en la distribución de la tensión, y los resultados de esta prueba respaldan la observación anterior de variaciones notables en las características centrales de la tensión entre los diferentes tipos de formulación.

In [ ]:
# Realiza la prueba de Kruskal-Wallis
kruskal.test(Tension ~ Formulación, data = datos)

### Revisar en la sesión

In [ ]:
# Realiza pruebas post hoc de Wilcoxon con ajuste de p-valores
pruebas_post_hoc <- pairwise.wilcox.test(datos$Tension, datos$`Formulación`, p.adj = "bonferroni")

# Muestra las pruebas post hoc
print(pruebas_post_hoc)

### Tensión por formulación agrunpando por carga 0

Para los datos en ausencia de carga, se puede observar que la formulación 33 presenta la menor tensión mediana. Mientras que la formulación 12 es quien presenta mayor tensión mediana.

In [ ]:
# Boxplot de Tension según la variable Formulación solo para carga 0
ggplot(datos_carga_0, aes(x = Formulación, y = Tension, fill = Formulación)) +
  geom_boxplot() +
  labs(title = "Boxplot de Tension según la Formulación (Carga 0)", x = "Formulación", y = "Tension") +
  theme_minimal() +
  theme(axis.text = element_text(size = 12), axis.title = element_text(size = 14))

La prueba de Kruskal-Wallis realizada para los datos sin carga, segmentados por las diferentes formulaciones (códigos numéricos 11, 12, 13, 21, 22, 23, 31, 32, 33), muestra un estadístico de chi-cuadrado de 2546.2 con 8 grados de libertad y un p-valor extremadamente bajo (< 2.2e-16). Este resultado indica que hay diferencias significativas en las medianas de la variable Tension entre al menos dos de los grupos de formulaciones para los casos sin carga. En resumen, la formulación sigue teniendo un impacto significativo en la distribución de la tensión incluso cuando no hay carga, según los resultados de esta prueba no paramétrica.

In [ ]:
# Realiza la prueba de Kruskal-Wallis
kruskal.test(Tension ~ Formulación, data = datos_carga_0)

### Revisar en la sesión

In [ ]:
# Realiza pruebas post hoc de Wilcoxon con ajuste de p-valores
pruebas_post_hoc0 <- pairwise.wilcox.test(datos$Tension, datos_carga_0$`Formulación`, p.adj = "bonferroni")

# Muestra las pruebas post hoc
print(pruebas_post_hoc0)

### Tensión por formulación agrunpando por carga 1

In [ ]:
# Boxplot de Tension según la variable Formulación solo para carga 1
ggplot(datos_carga_1, aes(x = Formulación, y = Tension, fill = Formulación)) +
  geom_boxplot() +
  labs(title = "Boxplot de Tension según la Formulación (Carga 1)", x = "Formulación", y = "Tension") +
  theme_minimal() +
  theme(axis.text = element_text(size = 12), axis.title = element_text(size = 14))


La prueba de Kruskal-Wallis realizada para los datos con carga, desglosados por las diferentes formulaciones (códigos numéricos 11, 12, 13, 21, 22, 23, 31, 32, 33), revela un estadístico de chi-cuadrado de 785.86 con 8 grados de libertad y un p-valor extremadamente bajo (< 2.2e-16). Estos resultados indican que hay diferencias significativas en las medianas de la variable Tension entre al menos dos de los grupos de formulaciones para los casos con carga. En otras palabras, la formulación sigue teniendo un impacto significativo en la distribución de la tensión incluso cuando hay carga, según los resultados de esta prueba no paramétrica.

In [ ]:
# Realiza la prueba de Kruskal-Wallis
kruskal.test(Tension ~ Formulación, data = datos_carga_1)

In [ ]:
# Realiza pruebas post hoc de Wilcoxon con ajuste de p-valores
pruebas_post_hoc1 <- pairwise.wilcox.test(datos$Tension, datos_carga_1$`Formulación`, p.adj = "bonferroni")

# Muestra las pruebas post hoc
print(pruebas_post_hoc1)

## Estadísticas Bivariadas

1. **Force y Distance:**
   - Correlación: 0.8516
   - Interpretación: Hay una correlación positiva fuerte (0.8516) entre las variables Force y Distance. Esto sugiere que a medida que una de las variables aumenta, la otra tiende a aumentar también.

2. **Force y Deformacion:**
   - Correlación: 0.8516
   - Interpretación: Existe una correlación positiva fuerte (0.8516) entre Force y Deformacion, indicando que ambas variables están positivamente relacionadas linealmente.

3. **Force y Tension:**
   - Correlación: 0.9989
   - Interpretación: Hay una correlación extremadamente fuerte (0.9989) entre Force y Tension. Esto sugiere una relación casi perfectamente positiva lineal entre estas dos variables.

4. **Distance y Deformacion:**
   - Correlación: 1.0000
   - Interpretación: Existe una correlación positiva perfecta (1.0000) entre Distance y Deformacion. Esto indica una relación lineal perfecta positiva entre estas dos variables.

5. **Distance y Tension:**
   - Correlación: 0.8436
   - Interpretación: Hay una correlación positiva fuerte (0.8436) entre Distance y Tension. A medida que una de las variables aumenta, la otra tiende a aumentar también.

6. **Deformacion y Tension:**
   - Correlación: 0.8436
   - Interpretación: Existe una correlación positiva fuerte (0.8436) entre Deformacion y Tension, indicando que ambas variables están positivamente relacionadas linealmente.


In [ ]:
# Instalar y cargar el paquete Hmisc
install.packages("Hmisc")
library(Hmisc)


In [ ]:
# Convertir todas las columnas a tipo numérico
columnas_interes <- as.data.frame(sapply(columnas_interes, as.numeric))

# Tratar valores faltantes
columnas_interes <- na.omit(columnas_interes)

# Calcular la matriz de correlación y p-valores
cor_test_result <- rcorr(as.matrix(columnas_interes))

# Obtener la matriz de correlación
correlacion_matrix <- cor_test_result$r

# Obtener la matriz de p-valores
p_values <- cor_test_result$P

# Imprimir la matriz de correlación
cat("Matriz de correlación:\n")
print(correlacion_matrix)

# Imprimir la matriz de p-valores
cat("\nP-valores:\n")
print(p_values)

In [ ]:
# Calcular la matriz de correlación
correlacion_matrix <- cor(columnas_interes)

# Calcular los p-valores
p_values <- psych::corr.test(columnas_interes)$p

# Gráfico de matriz de correlación con p-valores
corrplot(correlacion_matrix, method = "circle", p.mat = p_values, sig.level = 0.05)

In [ ]:
# Gráfico de dispersión con líneas de tendencia usando pairs()
pairs(columnas_interes, pch = 1, main = "Gráficos de Dispersión",
      col = "darkblue", cex.labels = 1.5, cex.axis = 1.5, cex.main = 2)
